In [1]:
import os
import dotenv
import requests

dotenv.load_dotenv()  # Load environment variables from .env file if it exists
# Ensure required environment variables are set
if not os.path.exists('.env'):
    print("Warning: .env file not found. Please create one with your credentials.")


# Load credentials from environment variables
CLIENT_ID = os.environ.get('GOVWIN_CLIENT_ID')
CLIENT_SECRET = os.environ.get('GOVWIN_CLIENT_SECRET')
USERNAME = os.environ.get('GOVWIN_USERNAME')
PASSWORD = os.environ.get('GOVWIN_PASSWORD')

# Check if all credentials are present
if not all([CLIENT_ID, CLIENT_SECRET, USERNAME, PASSWORD]):
    print("Error: Missing credentials. Please set environment variables:")
    print("  GOVWIN_CLIENT_ID, GOVWIN_CLIENT_SECRET, GOVWIN_USERNAME, GOVWIN_PASSWORD")
    exit(1)

# API endpoints
BASE_URL = "https://services.govwin.com"
TOKEN_URL = f"{BASE_URL}/neo-ws/oauth/token"

# Get access token
print("Testing connection to GovWin API...")
response = requests.post(
    TOKEN_URL,
    data={
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': 'password',
        'username': USERNAME,
        'password': PASSWORD,
        'scope': 'read'
    },
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
)

if response.status_code == 200:
    token_data = response.json()
    access_token = token_data.get('access_token')
    print("✓ Successfully connected to GovWin API!")
    print(f"✓ Access token received (expires in {token_data.get('expires_in')} seconds)")
    
    # Test a simple API call
    test_response = requests.get(
        f"{BASE_URL}/neo-ws/opportunities",
        headers={'Authorization': f'Bearer {access_token}'},
        params={'max': 1}
    )
    
    if test_response.status_code == 200:
        print("✓ API call successful!")
    else:
        print(f"✗ API call failed: {test_response.status_code}")
else:
    print(f"✗ Authentication failed: {response.status_code}")
    print(f"Error: {response.text}")

Testing connection to GovWin API...
✓ Successfully connected to GovWin API!
✓ Access token received (expires in 43199 seconds)
✓ API call successful!


In [18]:
"""
search_personnel_security.py
Search GovWin opportunities that match:
    • Keyword    : "personnel security"
    • NAICS      : 541611, 561611, 541214
    • PSC / PSCs : R408, R703
Results are returned as JSON; the first page (max-100) is printed.
"""
import os
import sys
import time
import dotenv
import requests

dotenv.load_dotenv()                     # reads .env for the four creds
BASE_URL   = "https://services.govwin.com"
TOKEN_URL  = f"{BASE_URL}/neo-ws/oauth/token"
SEARCH_URL = f"{BASE_URL}/neo-ws/opportunities"

def get_access_token() -> str:
    """Password-grant OAuth2 (valid 12 h)."""
    rsp = requests.post(
        TOKEN_URL,
        data={
            "client_id":     os.getenv("GOVWIN_CLIENT_ID"),
            "client_secret": os.getenv("GOVWIN_CLIENT_SECRET"),
            "grant_type":    "password",
            "username":      os.getenv("GOVWIN_USERNAME"),
            "password":      os.getenv("GOVWIN_PASSWORD"),
            "scope":         "read",
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        timeout=20,
    )
    if rsp.status_code != 200:
        sys.exit(f"Auth failed {rsp.status_code}: {rsp.text}")
    token = rsp.json()
    expires = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()+token['expires_in']))
    print(f"✓ token OK – expires {expires}")
    return token["access_token"]

def search_opportunities(token: str):
    params = {
        # ---- search criteria ----
        "q": '"personnel security"',                # exact phrase search
        "max": 100,     # up to 100 per page (default 10) :contentReference[oaicite:2]{index=2}
        "offset": 0,
    }

    rsp = requests.get(
        SEARCH_URL,
        headers={"Authorization": f"Bearer {token}"},
        params=params,
        timeout=20,
    )
    if rsp.status_code != 200:
        sys.exit(f"Search failed {rsp.status_code}: {rsp.text}")

    data = rsp.json()
    print(f"Found {data['meta']['paging']['totalCount']} matching opportunities\n")
    # pretty-print the first few titles
    for opp in data.get("opportunities", [])[:5]:
        print(f"{opp['id']:>10}  {opp['title']}")
    # full JSON is still in `data`

if __name__ == "__main__":
    access_token = get_access_token()
    search_opportunities(access_token)


✓ token OK – expires 2025-06-22 08:05:02
Found 3 matching opportunities

 OPP253641  PERSONNEL SECURITY SUPPORT SERVICES
 OPP253037  INFORMATION TECHNOLOGY ARTIFICIAL INTELLIGENCE MACHINE LEARNING PROGRAMMATIC SUPPORT SERVICES (IT)(AI)
FBO4026782  Multi-Function Device (MFD) Lease Agreement and PM; to include: Lease, Maintenance, and Overages.


In [7]:
'GSA eBuy tests'

import os, time, requests, dotenv, datetime as dt
from pprint import pprint              # nice console preview

dotenv.load_dotenv()                   # GOVWIN_* creds in .env
BASE_URL   = "https://services.govwin.com"
TOKEN_URL  = f"{BASE_URL}/neo-ws/oauth/token"
SEARCH_URL = f"{BASE_URL}/neo-ws/opportunities"

def get_access_token() -> str:
    rsp = requests.post(
        TOKEN_URL,
        data={
            "client_id":     os.getenv("GOVWIN_CLIENT_ID"),
            "client_secret": os.getenv("GOVWIN_CLIENT_SECRET"),
            "grant_type":    "password",
            "username":      os.getenv("GOVWIN_USERNAME"),
            "password":      os.getenv("GOVWIN_PASSWORD"),
            "scope":         "read",
        },
        timeout=20,
    )
    rsp.raise_for_status()
    token = rsp.json()
    expires = time.strftime("%Y-%m-%d %H:%M:%S",
                            time.localtime(time.time() + token["expires_in"]))
    print(f"✓ token OK – expires {expires}")
    return token["access_token"]

def search_tns(token: str,
               keyword: str = '"personnel "',
               lookback_days: int = 10,
               max_per_page: int = 100):
    date_from = (dt.datetime.utcnow() - dt.timedelta(days=lookback_days)).strftime("%Y-%m-%d")
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "q": keyword,
        "market": "Federal",
        "oppType": "TNS",               # ← Task-Order Notices
        "oppSelectionDateFrom": date_from,
        "max": max_per_page,
        "offset": 0,
    }

    all_rows = []
    while True:
        rsp = requests.get(SEARCH_URL, headers=headers, params=params, timeout=20)
        rsp.raise_for_status()
        page = rsp.json().get("opportunities", [])
        all_rows.extend(page)

        print(f"Fetched {len(page):3d} rows  (offset {params['offset']})")
        if len(page) < max_per_page:           # last page reached
            break
        params["offset"] += len(page)

    print(f"\nTotal TNS rows returned: {len(all_rows)}")
    for opp in all_rows[:5]:
        print(f"{opp['id']:>10}  {opp['title'][:90]}")

    return all_rows

if __name__ == "__main__":
    token = get_access_token()
    tns_data = search_tns(token)


✓ token OK – expires 2025-06-22 08:05:03
Fetched   1 rows  (offset 0)

Total TNS rows returned: 1
TNS2244465  ADMINISTRATIVE PERSONNEL INVESTIGATIONS MISCONDUCT AND/OR HARASSMENT (HR) - BPA


In [16]:
'Tracked Opportunities'

import os, time, requests, dotenv, datetime as dt
from pprint import pprint              # optional pretty preview

dotenv.load_dotenv()                   # GOVWIN_* creds in .env
BASE_URL   = "https://services.govwin.com"
TOKEN_URL  = f"{BASE_URL}/neo-ws/oauth/token"
SEARCH_URL = f"{BASE_URL}/neo-ws/opportunities"


def get_access_token() -> str:
    rsp = requests.post(
        TOKEN_URL,
        data={
            "client_id":     os.getenv("GOVWIN_CLIENT_ID"),
            "client_secret": os.getenv("GOVWIN_CLIENT_SECRET"),
            "grant_type":    "password",
            "username":      os.getenv("GOVWIN_USERNAME"),
            "password":      os.getenv("GOVWIN_PASSWORD"),
            "scope":         "read",
        },
        timeout=20,
    )
    rsp.raise_for_status()
    tok = rsp.json()
    print("✓ token OK – expires", time.strftime(
        "%Y-%m-%d %H:%M:%S",
        time.localtime(time.time() + tok["expires_in"])
    ))
    return tok["access_token"]


def search_tracked(
        token: str,
        keyword: str = '"personnel"',
        lookback_days: int = 9,
        max_per_page: int = 100,
    ):
    """Fetch analyst-curated tracked opportunities (oppType=OPP)."""
    date_from = (dt.datetime.utcnow() - dt.timedelta(days=lookback_days)
                ).strftime("%Y-%m-%d")

    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "q": keyword,
        "market": "Federal",       # drop if you need state-&-local too
        "oppType": "TNS,OPP",          # ← tracked opps
        "oppSelectionDateFrom": date_from,
        "max": max_per_page,
        "offset": 0,
    }

    rows = []
    while True:
        rsp = requests.get(SEARCH_URL, headers=headers,
                           params=params, timeout=20)
        rsp.raise_for_status()
        page = rsp.json().get("opportunities", [])
        rows.extend(page)

        print(f"Fetched {len(page):3d}  (offset {params['offset']})")
        if len(page) < max_per_page:
            break
        params["offset"] += len(page)

    print(f"\nTotal OPP rows returned: {len(rows)}")
    for opp in rows:
        print(f"{opp['id']:>10}  {opp['title'][:90]}")
    return rows


if __name__ == "__main__":
    tok = get_access_token()
    opp_data = search_tracked(tok)


✓ token OK – expires 2025-06-22 08:05:03
Fetched 100  (offset 0)
Fetched 100  (offset 100)
Fetched 100  (offset 200)
Fetched  95  (offset 300)

Total OPP rows returned: 395
 OPP250776  COBOL DATABASE MODERNIZATION
 OPP253641  PERSONNEL SECURITY SUPPORT SERVICES
 OPP222941  DEFENSE READINESS REPORTING SYSTEM STRATEGIC MODERNIZATION (DRRS-S)
 OPP255678  DCPAS FITNESS CENTER AND HEALTH PROMOTION SERVICES
 OPP242553  MILITARY COMMUNITY AND FAMILY POLICY OUTREACH  DIGITAL ENTERPRISE SERVICES III (MCFP)(MODE
 OPP246272  FY25 CONTRACTOR SUPPORT PERSONNEL PM BM AND PA
 OPP224890  FACILITY ENGINEERING OPERATIONS AND MAINTENANCE AND CONFERENCE SUPPORT SERVICES FOR FEDERA
 OPP201927  CENSUS INTEGRATED PERSONNEL AND PAYROLL SYSTEM (CIPPS)
 OPP244722  DPAC MARKET RESEARCH AND STUDIES SURVEY SERVICES
 OPP241524  TAILORING SERVICES FOR ACCESSION PERSONNEL USCG TRAINING CENTER CAPE MAY NJ
 OPP227404  NATIONAL YOUTH CHALLENGE PROGRAM AND THE DEPARTMENT OF DEFENSE STARBASE PROGRAM
 OPP225841  DYNAMIC AU

In [8]:
"""
GovWin Personnel Security Opportunities API Tool
Retrieves opportunities matching "personnel security" from the last 24 hours
with comprehensive data extraction and error handling.
"""
import os
import sys
import time
import json
import dotenv
import requests
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any

dotenv.load_dotenv()

class GovWinAPI:
    def __init__(self):
        self.base_url = "https://services.govwin.com"
        self.token_url = f"{self.base_url}/neo-ws/oauth/token"
        self.search_url = f"{self.base_url}/neo-ws/opportunities"
        self.access_token = None
        self.token_expiry = None
        
    def get_access_token(self) -> str:
        """Get OAuth2 access token (valid for 12 hours)."""
        if self.access_token and self.token_expiry and time.time() < self.token_expiry:
            return self.access_token
            
        try:
            response = requests.post(
                self.token_url,
                data={
                    "client_id": os.getenv("GOVWIN_CLIENT_ID"),
                    "client_secret": os.getenv("GOVWIN_CLIENT_SECRET"),
                    "grant_type": "password",
                    "username": os.getenv("GOVWIN_USERNAME"),
                    "password": os.getenv("GOVWIN_PASSWORD"),
                    "scope": "read",
                },
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                timeout=20,
            )
            
            if response.status_code != 200:
                raise Exception(f"Auth failed {response.status_code}: {response.text}")
                
            token_data = response.json()
            self.access_token = token_data["access_token"]
            self.token_expiry = time.time() + token_data["expires_in"] - 300  # 5 min buffer
            
            expires_str = time.strftime("%Y-%m-%d %H:%M:%S", 
                                      time.localtime(self.token_expiry))
            print(f"✓ Token obtained - expires {expires_str}")
            
            return self.access_token
            
        except Exception as e:
            print(f"❌ Authentication failed: {e}")
            sys.exit(1)
    
    def safe_get(self, data: Dict, path: str, default: Any = None) -> Any:
        """Safely get nested dictionary values."""
        try:
            keys = path.split('.')
            result = data
            for key in keys:
                if isinstance(result, dict) and key in result:
                    result = result[key]
                else:
                    return default
            return result
        except:
            return default
    
    def extract_opportunity_data(self, opp: Dict) -> Dict:
        """Extract all relevant data from an opportunity, handling missing fields gracefully."""
        extracted = {
            # Basic Information
            "opportunity_id": self.safe_get(opp, "id"),
            "iq_opp_id": self.safe_get(opp, "iqOppId"),
            "title": self.safe_get(opp, "title"),
            "status": self.safe_get(opp, "status"),
            "type": self.safe_get(opp, "type"),
            "internal_status": self.safe_get(opp, "internalStatus"),
            
            # Dates
            "created_date": self.safe_get(opp, "createdDate"),
            "update_date": self.safe_get(opp, "updateDate"),
            "solicitation_date": self.safe_get(opp, "solicitationDate.value"),
            "response_date": self.safe_get(opp, "responseDate"),
            "award_date": self.safe_get(opp, "awardDate"),
            "termination_date": self.safe_get(opp, "terminationDateTx"),
            
            # Solicitation Information
            "solicitation_number": self.safe_get(opp, "solicitationNumber"),
            "source_url": self.safe_get(opp, "sourceURL"),
            
            # Financial Information
            "opp_value": self.safe_get(opp, "oppValue"),
            "opp_value_canada": self.safe_get(opp, "oppValueCanada"),
            "plan_price_tx": self.safe_get(opp, "planPriceTx"),
            "plan_price_canadian": self.safe_get(opp, "planPriceCanadian"),
            "fed_prime_obligation_amt": self.safe_get(opp, "fedPrimeObligationAmt"),
            "revenue": self.safe_get(opp, "revenue"),
            
            # Government Entity
            "gov_entity_id": self.safe_get(opp, "govEntity.id"),
            "gov_entity_title": self.safe_get(opp, "govEntity.title"),
            "gov_type": self.safe_get(opp, "govType"),
            
            # Classification and Requirements
            "primary_naics_id": self.safe_get(opp, "primaryNAICS.id"),
            "primary_naics_title": self.safe_get(opp, "primaryNAICS.title"),
            "primary_naics_size_standard": self.safe_get(opp, "primaryNAICS.sizeStandard"),
            "classification_code_desc": self.safe_get(opp, "classificationCodeDesc"),
            "primary_requirement": self.safe_get(opp, "primaryRequirement"),
            "procurement": self.safe_get(opp, "procurement"),
            
            # Competition and Contract Information
            "competition_types": [],
            "contract_types": [],
            "additional_naics": [],
            
            # Location and Performance
            "country": self.safe_get(opp, "country"),
            "city": self.safe_get(opp, "city"),
            "zip": self.safe_get(opp, "zip"),
            
            # Description and Duration
            "description": self.safe_get(opp, "description"),
            "duration": self.safe_get(opp, "duration"),
            
            # Security and Business Requirements
            "capabilities": self.safe_get(opp, "capabilities"),
            "cage_code": self.safe_get(opp, "cageCode"),
            "cmmc_requirements": self.safe_get(opp, "cmmcRequirements"),
            "socio_economic_status": self.safe_get(opp, "socioEconomicStatus"),
            "small_bus_naics": self.safe_get(opp, "smallBusNaics"),
            "org_certification": self.safe_get(opp, "orgCertification"),
            "org_facility_clearance": self.safe_get(opp, "orgFacilityClearance"),
            "org_staff_clearance": self.safe_get(opp, "orgStaffClearance"),
            
            # Estimates and Flags
            "is_deltek_estimate": self.safe_get(opp, "isDeltekEstimate"),
            "priority": self.safe_get(opp, "priority"),
            
            # Award Information
            "type_of_award": self.safe_get(opp, "typeOfAward"),
            "exclusion_type": self.safe_get(opp, "exclusionTypeTx"),
            
            # Partner Information
            "partner_response_date": self.safe_get(opp, "partnerResponseDate"),
            
            # Links
            "web_href": self.safe_get(opp, "links.webHref.href"),
        }
        
        # Extract competition types
        comp_types = self.safe_get(opp, "competitionTypes", [])
        if comp_types:
            extracted["competition_types"] = [
                {
                    "id": self.safe_get(comp, "id"),
                    "title": self.safe_get(comp, "title")
                }
                for comp in comp_types
            ]
        
        # Extract contract types
        contract_types = self.safe_get(opp, "contractTypes", [])
        if contract_types:
            extracted["contract_types"] = [
                {
                    "id": self.safe_get(cont, "id"),
                    "title": self.safe_get(cont, "title")
                }
                for cont in contract_types
            ]
        
        # Extract additional NAICS
        add_naics = self.safe_get(opp, "additionalNaics", [])
        if add_naics:
            extracted["additional_naics"] = [
                {
                    "id": self.safe_get(naics, "id"),
                    "title": self.safe_get(naics, "title"),
                    "size_standard": self.safe_get(naics, "sizeStandard")
                }
                for naics in add_naics
            ]
        
        return extracted
    
    def search_personnel_security_opportunities(self, 
                                              last_hours: int = 24,
                                              max_results: int = 100,
                                              include_naics: List[str] = None,
                                              include_psc: List[str] = None) -> Dict:
        """
        Search for personnel security opportunities from the last N hours.
        
        Args:
            last_hours: Number of hours to look back (default: 24)
            max_results: Maximum results per page (default: 100, max: 100)
            include_naics: Optional list of NAICS codes to include
            include_psc: Optional list of PSC codes to include
        """
        token = self.get_access_token()
        
        # Build search parameters
        params = {
            "q": '"personnel security"',  # Exact phrase search
            "oppSelectionDateFrom": f"-{last_hours}H",  # Relative date format
            "oppCategory": 2,  # New and Update
            "max": min(max_results, 100),  # API limit is 100
            "offset": 0,
        }
        
        # Add optional NAICS codes
        if include_naics:
            params["naics"] = ",".join(include_naics)
        
        # Add optional PSC codes  
        if include_psc:
            params["classificationCodes"] = ",".join(include_psc)
        
        try:
            response = requests.get(
                self.search_url,
                headers={"Authorization": f"Bearer {token}"},
                params=params,
                timeout=30,
            )
            
            if response.status_code != 200:
                raise Exception(f"Search failed {response.status_code}: {response.text}")
            
            data = response.json()
            total_count = self.safe_get(data, "meta.paging.totalCount", 0)
            
            print(f"✓ Found {total_count} personnel security opportunities")
            
            # Extract detailed data from each opportunity
            opportunities = []
            for opp in data.get("opportunities", []):
                extracted_opp = self.extract_opportunity_data(opp)
                opportunities.append(extracted_opp)
            
            return {
                "search_metadata": {
                    "search_time": datetime.now().isoformat(),
                    "total_count": total_count,
                    "returned_count": len(opportunities),
                    "search_parameters": params,
                    "hours_back": last_hours
                },
                "opportunities": opportunities
            }
            
        except Exception as e:
            print(f"❌ Search failed: {e}")
            return {
                "search_metadata": {
                    "search_time": datetime.now().isoformat(),
                    "error": str(e),
                    "search_parameters": params
                },
                "opportunities": []
            }
    
    def get_all_recent_opportunities(self, last_hours: int = 24) -> List[Dict]:
        """
        Get all personnel security opportunities from the last N hours,
        handling pagination if necessary.
        """
        all_opportunities = []
        offset = 0
        max_per_page = 100
        
        while True:
            token = self.get_access_token()
            
            params = {
                "q": '"personnel security"',
                "oppSelectionDateFrom": f"-{last_hours}H",
                "oppCategory": 2,
                "max": max_per_page,
                "offset": offset,
            }
            
            try:
                response = requests.get(
                    self.search_url,
                    headers={"Authorization": f"Bearer {token}"},
                    params=params,
                    timeout=30,
                )
                
                if response.status_code != 200:
                    print(f"❌ Pagination request failed: {response.status_code}")
                    break
                
                data = response.json()
                opportunities = data.get("opportunities", [])
                
                if not opportunities:
                    break
                
                # Extract data from this page
                for opp in opportunities:
                    extracted_opp = self.extract_opportunity_data(opp)
                    all_opportunities.append(extracted_opp)
                
                # Check if we got all results
                total_count = self.safe_get(data, "meta.paging.totalCount", 0)
                if len(all_opportunities) >= total_count:
                    break
                    
                offset += max_per_page
                print(f"📄 Retrieved {len(all_opportunities)}/{total_count} opportunities...")
                
                # Add a small delay to respect rate limits
                time.sleep(0.1)
                
            except Exception as e:
                print(f"❌ Pagination error: {e}")
                break
        
        return all_opportunities

def main():
    """Main execution function for testing."""
    api = GovWinAPI()
    
    print("🔍 Searching for personnel security opportunities from last 24 hours...")
    
    # Get opportunities with common NAICS codes for personnel security
    personnel_security_naics = [
        "541611",  # Administrative Management and General Management Consulting Services
        "561611",  # Investigation, Guard, and Armored Car Services
        "541214",  # Payroll Services
    ]
    
    personnel_security_psc = [
        "R408",  # Support- Professional: Program Management/Support Services
        "R703",  # Support- Professional: Other Safety and Security Services
    ]
    
    result = api.search_personnel_security_opportunities(
        last_hours=24,
        max_results=100,
        # include_naics=personnel_security_naics,
        # include_psc=personnel_security_psc
    )
    
    # Print summary
    print(f"\n📊 Search Summary:")
    print(f"   Total opportunities found: {result['search_metadata'].get('total_count', 0)}")
    print(f"   Opportunities retrieved: {result['search_metadata'].get('returned_count', 0)}")
    
    # Print first few opportunities with key details
    if result["opportunities"]:
        print(f"\n📋 Sample Opportunities:")
        for i, opp in enumerate(result["opportunities"][:5], 1):
            print(f"\n   {i}. {opp.get('title', 'No Title')}")
            print(f"      ID: {opp.get('opportunity_id', 'N/A')}")
            print(f"      Status: {opp.get('status', 'N/A')}")
            print(f"      Value: ${opp.get('opp_value', 'N/A')}")
            print(f"      Gov Entity: {opp.get('gov_entity_title', 'N/A')}")
            print(f"      Solicitation #: {opp.get('solicitation_number', 'N/A')}")
            if opp.get('solicitation_date'):
                print(f"      Solicitation Date: {opp.get('solicitation_date', 'N/A')}")
            if opp.get('response_date'):
                print(f"      Response Date: {opp.get('response_date', 'N/A')}")
    
    # Save to JSON file with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"personnel_security_opportunities_{timestamp}.json"
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False, default=str)
    
    print(f"\n💾 Results saved to: {filename}")
    
    return result

if __name__ == "__main__":
    # Example usage for serverless function
    result = main()

🔍 Searching for personnel security opportunities from last 24 hours...
✓ Token obtained - expires 2025-06-12 02:55:53
✓ Found 7 personnel security opportunities

📊 Search Summary:
   Total opportunities found: 7
   Opportunities retrieved: 7

📋 Sample Opportunities:

   1. FIELD BACKGROUND INVESTIGATIVE SERVICES
      ID: OPP255515
      Status: Post-RFP
      Value: $0
      Gov Entity: OFFICE OF PROFESSIONAL RESPONSIBILITY AND SECURITY OPERATIONS
      Solicitation #: 15A00025R00000040
      Solicitation Date: 2025-06-11T00:00:00.000

   2. SECURITY PROCESSING SUPPORT SERVICES (SPSS)
      ID: OPP248848
      Status: Pre-RFP
      Value: $0
      Gov Entity: ASST SECRETARY FOR ADMINISTRATION
      Solicitation #: 75P00125R00002
      Solicitation Date: 2025-09-30T00:00:00.000

   3. ICE OFFICE OF PROFESSIONAL RESPONSIBILITY TECHNOLOGY SOLUTIONS
      ID: OPP245230
      Status: Pre-RFP
      Value: $0
      Gov Entity: IMMIGRATION AND CUSTOMS ENFORCEMENT
      Solicitation #: 
      

In [10]:
"""
GovWin Personnel Security Opportunities API Tool
Retrieves opportunities matching "personnel security" from the last 24 hours
with comprehensive data extraction and error handling.
"""
import os
import sys
import time
import json
import dotenv
import requests
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any

dotenv.load_dotenv()

class GovWinAPI:
    def __init__(self):
        self.base_url = "https://services.govwin.com"
        self.token_url = f"{self.base_url}/neo-ws/oauth/token"
        self.search_url = f"{self.base_url}/neo-ws/opportunities"
        self.access_token = None
        self.token_expiry = None
        
    def get_access_token(self) -> str:
        """Get OAuth2 access token (valid for 12 hours)."""
        if self.access_token and self.token_expiry and time.time() < self.token_expiry:
            return self.access_token
            
        try:
            response = requests.post(
                self.token_url,
                data={
                    "client_id": os.getenv("GOVWIN_CLIENT_ID"),
                    "client_secret": os.getenv("GOVWIN_CLIENT_SECRET"),
                    "grant_type": "password",
                    "username": os.getenv("GOVWIN_USERNAME"),
                    "password": os.getenv("GOVWIN_PASSWORD"),
                    "scope": "read",
                },
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                timeout=20,
            )
            
            if response.status_code != 200:
                raise Exception(f"Auth failed {response.status_code}: {response.text}")
                
            token_data = response.json()
            self.access_token = token_data["access_token"]
            self.token_expiry = time.time() + token_data["expires_in"] - 300  # 5 min buffer
            
            expires_str = time.strftime("%Y-%m-%d %H:%M:%S", 
                                      time.localtime(self.token_expiry))
            print(f"✓ Token obtained - expires {expires_str}")
            
            return self.access_token
            
        except Exception as e:
            print(f"❌ Authentication failed: {e}")
            sys.exit(1)
    
    def safe_get(self, data: Dict, path: str, default: Any = None) -> Any:
        """Safely get nested dictionary values."""
        try:
            keys = path.split('.')
            result = data
            for key in keys:
                if isinstance(result, dict) and key in result:
                    result = result[key]
                else:
                    return default
            return result
        except:
            return default
    
    def get_source_platform(self, opp_type: str) -> str:
        """Determine the source platform based on opportunity type."""
        platform_mapping = {
            "FBO": "SAM.gov",
            "OPP": "Deltek Tracked",
            "TNS": "Task Orders (TOONS)",
            "BID": "State/Local Bids",
            "TOP": "Opportunity Manager"
        }
        return platform_mapping.get(opp_type, "Unknown")
    
    def is_sam_gov_opportunity(self, opp: Dict) -> bool:
        """Check if opportunity is from SAM.gov."""
        return self.safe_get(opp, "type") == "FBO"
    
    def extract_opportunity_data(self, opp: Dict) -> Dict:
        """Extract all relevant data from an opportunity, handling missing fields gracefully."""
        # Determine source platform
        opp_type = self.safe_get(opp, "type")
        source_platform = self.get_source_platform(opp_type)
        
        extracted = {
            # Basic Information
            "opportunity_id": self.safe_get(opp, "id"),
            "iq_opp_id": self.safe_get(opp, "iqOppId"),
            "title": self.safe_get(opp, "title"),
            "status": self.safe_get(opp, "status"),
            "type": opp_type,
            "source_platform": source_platform,
            "internal_status": self.safe_get(opp, "internalStatus"),
            
            # Dates
            "created_date": self.safe_get(opp, "createdDate"),
            "update_date": self.safe_get(opp, "updateDate"),
            "solicitation_date": self.safe_get(opp, "solicitationDate.value"),
            "response_date": self.safe_get(opp, "responseDate"),
            "award_date": self.safe_get(opp, "awardDate"),
            "termination_date": self.safe_get(opp, "terminationDateTx"),
            
            # Solicitation Information
            "solicitation_number": self.safe_get(opp, "solicitationNumber"),
            "source_url": self.safe_get(opp, "sourceURL"),
            
            # Financial Information
            "opp_value": self.safe_get(opp, "oppValue"),
            "opp_value_canada": self.safe_get(opp, "oppValueCanada"),
            "plan_price_tx": self.safe_get(opp, "planPriceTx"),
            "plan_price_canadian": self.safe_get(opp, "planPriceCanadian"),
            "fed_prime_obligation_amt": self.safe_get(opp, "fedPrimeObligationAmt"),
            "revenue": self.safe_get(opp, "revenue"),
            
            # Government Entity
            "gov_entity_id": self.safe_get(opp, "govEntity.id"),
            "gov_entity_title": self.safe_get(opp, "govEntity.title"),
            "gov_type": self.safe_get(opp, "govType"),
            
            # Classification and Requirements
            "primary_naics_id": self.safe_get(opp, "primaryNAICS.id"),
            "primary_naics_title": self.safe_get(opp, "primaryNAICS.title"),
            "primary_naics_size_standard": self.safe_get(opp, "primaryNAICS.sizeStandard"),
            "classification_code_desc": self.safe_get(opp, "classificationCodeDesc"),
            "primary_requirement": self.safe_get(opp, "primaryRequirement"),
            "procurement": self.safe_get(opp, "procurement"),
            
            # Competition and Contract Information
            "competition_types": [],
            "contract_types": [],
            "additional_naics": [],
            
            # Location and Performance
            "country": self.safe_get(opp, "country"),
            "city": self.safe_get(opp, "city"),
            "zip": self.safe_get(opp, "zip"),
            
            # Description and Duration
            "description": self.safe_get(opp, "description"),
            "duration": self.safe_get(opp, "duration"),
            
            # Security and Business Requirements
            "capabilities": self.safe_get(opp, "capabilities"),
            "cage_code": self.safe_get(opp, "cageCode"),
            "cmmc_requirements": self.safe_get(opp, "cmmcRequirements"),
            "socio_economic_status": self.safe_get(opp, "socioEconomicStatus"),
            "small_bus_naics": self.safe_get(opp, "smallBusNaics"),
            "org_certification": self.safe_get(opp, "orgCertification"),
            "org_facility_clearance": self.safe_get(opp, "orgFacilityClearance"),
            "org_staff_clearance": self.safe_get(opp, "orgStaffClearance"),
            
            # Estimates and Flags
            "is_deltek_estimate": self.safe_get(opp, "isDeltekEstimate"),
            "priority": self.safe_get(opp, "priority"),
            
            # Award Information
            "type_of_award": self.safe_get(opp, "typeOfAward"),
            "exclusion_type": self.safe_get(opp, "exclusionTypeTx"),
            
            # Partner Information
            "partner_response_date": self.safe_get(opp, "partnerResponseDate"),
            
            # Links
            "web_href": self.safe_get(opp, "links.webHref.href"),
        }
        
        # Extract competition types
        comp_types = self.safe_get(opp, "competitionTypes", [])
        if comp_types:
            extracted["competition_types"] = [
                {
                    "id": self.safe_get(comp, "id"),
                    "title": self.safe_get(comp, "title")
                }
                for comp in comp_types
            ]
        
        # Extract contract types
        contract_types = self.safe_get(opp, "contractTypes", [])
        if contract_types:
            extracted["contract_types"] = [
                {
                    "id": self.safe_get(cont, "id"),
                    "title": self.safe_get(cont, "title")
                }
                for cont in contract_types
            ]
        
        # Extract additional NAICS
        add_naics = self.safe_get(opp, "additionalNaics", [])
        if add_naics:
            extracted["additional_naics"] = [
                {
                    "id": self.safe_get(naics, "id"),
                    "title": self.safe_get(naics, "title"),
                    "size_standard": self.safe_get(naics, "sizeStandard")
                }
                for naics in add_naics
            ]
        
        return extracted
    
    def search_personnel_security_opportunities(self, 
                                              last_hours: int = 24,
                                              max_results: int = 100,
                                              include_naics: List[str] = None,
                                              include_psc: List[str] = None) -> Dict:
        """
        Search for personnel security opportunities from the last N hours.
        
        Args:
            last_hours: Number of hours to look back (default: 24)
            max_results: Maximum results per page (default: 100, max: 100)
            include_naics: Optional list of NAICS codes to include
            include_psc: Optional list of PSC codes to include
        """
        token = self.get_access_token()
        
        # Build search parameters
        params = {
            "q": '"personnel security"',  # Exact phrase search
            "oppSelectionDateFrom": f"-{last_hours}H",  # Relative date format
            "oppCategory": 2,  # New and Update
            "max": min(max_results, 100),  # API limit is 100
            "offset": 0,
        }
        
        # Add optional NAICS codes
        if include_naics:
            params["naics"] = ",".join(include_naics)
        
        # Add optional PSC codes  
        if include_psc:
            params["classificationCodes"] = ",".join(include_psc)
        
        try:
            response = requests.get(
                self.search_url,
                headers={"Authorization": f"Bearer {token}"},
                params=params,
                timeout=30,
            )
            
            if response.status_code != 200:
                raise Exception(f"Search failed {response.status_code}: {response.text}")
            
            data = response.json()
            total_count = self.safe_get(data, "meta.paging.totalCount", 0)
            
            print(f"✓ Found {total_count} personnel security opportunities")
            
            # Extract detailed data from each opportunity
            opportunities = []
            for opp in data.get("opportunities", []):
                extracted_opp = self.extract_opportunity_data(opp)
                opportunities.append(extracted_opp)
            
            return {
                "search_metadata": {
                    "search_time": datetime.now().isoformat(),
                    "total_count": total_count,
                    "returned_count": len(opportunities),
                    "search_parameters": params,
                    "hours_back": last_hours
                },
                "opportunities": opportunities
            }
            
        except Exception as e:
            print(f"❌ Search failed: {e}")
            return {
                "search_metadata": {
                    "search_time": datetime.now().isoformat(),
                    "error": str(e),
                    "search_parameters": params
                },
                "opportunities": []
            }
    
    def get_all_recent_opportunities(self, last_hours: int = 24) -> List[Dict]:
        """
        Get all personnel security opportunities from the last N hours,
        handling pagination if necessary.
        """
        all_opportunities = []
        offset = 0
        max_per_page = 100
        
        while True:
            token = self.get_access_token()
            
            params = {
                "q": '"personnel security"',
                "oppSelectionDateFrom": f"-{last_hours}H",
                "oppCategory": 2,
                "max": max_per_page,
                "offset": offset,
            }
            
            try:
                response = requests.get(
                    self.search_url,
                    headers={"Authorization": f"Bearer {token}"},
                    params=params,
                    timeout=30,
                )
                
                if response.status_code != 200:
                    print(f"❌ Pagination request failed: {response.status_code}")
                    break
                
                data = response.json()
                opportunities = data.get("opportunities", [])
                
                if not opportunities:
                    break
                
                # Extract data from this page
                for opp in opportunities:
                    extracted_opp = self.extract_opportunity_data(opp)
                    all_opportunities.append(extracted_opp)
                
                # Check if we got all results
                total_count = self.safe_get(data, "meta.paging.totalCount", 0)
                if len(all_opportunities) >= total_count:
                    break
                    
                offset += max_per_page
                print(f"📄 Retrieved {len(all_opportunities)}/{total_count} opportunities...")
                
                # Add a small delay to respect rate limits
                time.sleep(0.1)
                
            except Exception as e:
                print(f"❌ Pagination error: {e}")
                break
        
        return all_opportunities

def main():
    """Main execution function for testing."""
    api = GovWinAPI()
    
    print("🔍 Searching for personnel security opportunities from last 24 hours...")
    
    # Get opportunities with common NAICS codes for personnel security
    personnel_security_naics = [
        "541611",  # Administrative Management and General Management Consulting Services
        "561611",  # Investigation, Guard, and Armored Car Services
        "541214",  # Payroll Services
    ]
    
    personnel_security_psc = [
        "R408",  # Support- Professional: Program Management/Support Services
        "R703",  # Support- Professional: Other Safety and Security Services
    ]
    
    result = api.search_personnel_security_opportunities(
        last_hours=24,
        max_results=100,
        # include_naics=personnel_security_naics,
        # include_psc=personnel_security_psc
    )
    
    # Print summary
    print(f"\n📊 Search Summary:")
    print(f"   Total opportunities found: {result['search_metadata'].get('total_count', 0)}")
    print(f"   Opportunities retrieved: {result['search_metadata'].get('returned_count', 0)}")
    
    # Print first few opportunities with key details
    if result["opportunities"]:
        print(f"\n📋 Sample Opportunities:")
        for i, opp in enumerate(result["opportunities"][:5], 1):
            print(f"\n   {i}. {opp.get('title', 'No Title')}")
            print(f"      ID: {opp.get('opportunity_id', 'N/A')}")
            print(f"      Source: {opp.get('source_platform', 'N/A')} (Type: {opp.get('type', 'N/A')})")
            print(f"      Status: {opp.get('status', 'N/A')}")
            print(f"      Value: ${opp.get('opp_value', 'N/A')}")
            print(f"      Gov Entity: {opp.get('gov_entity_title', 'N/A')}")
            print(f"      Solicitation #: {opp.get('solicitation_number', 'N/A')}")
            if opp.get('solicitation_date'):
                print(f"      Solicitation Date: {opp.get('solicitation_date', 'N/A')}")
            if opp.get('response_date'):
                print(f"      Response Date: {opp.get('response_date', 'N/A')}")
    
    # Print source breakdown
    if result["opportunities"]:
        source_counts = {}
        for opp in result["opportunities"]:
            source = opp.get('source_platform', 'Unknown')
            source_counts[source] = source_counts.get(source, 0) + 1
        
        print(f"\n📊 Source Platform Breakdown:")
        for source, count in source_counts.items():
            print(f"   {source}: {count} opportunities")
    
    # Save to JSON file with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"personnel_security_opportunities_{timestamp}.json"
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False, default=str)
    
    print(f"\n💾 Results saved to: {filename}")
    
    return result

if __name__ == "__main__":
    # Example usage for serverless function
    result = main()

🔍 Searching for personnel security opportunities from last 24 hours...
✓ Token obtained - expires 2025-06-12 02:55:53
✓ Found 7 personnel security opportunities

📊 Search Summary:
   Total opportunities found: 7
   Opportunities retrieved: 7

📋 Sample Opportunities:

   1. FIELD BACKGROUND INVESTIGATIVE SERVICES
      ID: OPP255515
      Source: Unknown (Type: trackedopp)
      Status: Post-RFP
      Value: $0
      Gov Entity: OFFICE OF PROFESSIONAL RESPONSIBILITY AND SECURITY OPERATIONS
      Solicitation #: 15A00025R00000040
      Solicitation Date: 2025-06-11T00:00:00.000

   2. SECURITY PROCESSING SUPPORT SERVICES (SPSS)
      ID: OPP248848
      Source: Unknown (Type: trackedopp)
      Status: Pre-RFP
      Value: $0
      Gov Entity: ASST SECRETARY FOR ADMINISTRATION
      Solicitation #: 75P00125R00002
      Solicitation Date: 2025-09-30T00:00:00.000

   3. ICE OFFICE OF PROFESSIONAL RESPONSIBILITY TECHNOLOGY SOLUTIONS
      ID: OPP245230
      Source: Unknown (Type: trackedopp